In [4]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.signal import argrelextrema
import time

from math import *

In [5]:
W=2*np.pi
T=(2*np.pi)/W
n=50
L=n*T

N_grid=20000
t=np.linspace(0,L,N_grid)
h=t[1]-t[0]
p=int(round(T/h))

d=2.5
dd=-2.45

dw_samples=np.linspace(0.05, 1.25, 200)
dual_border=[]

In [6]:
for dw in dw_samples:
    left=1
    right=N_grid-2*p-1
 
    start = time.time()
    while (right-left>1):
        s=round((left+right)/2)

        k0=np.array([d]*N_grid)
        k0[2*p:2*p+s]=d+dd
        k0_f=interpolate.interp1d(t, k0, bounds_error=False, fill_value="extrapolate")

        f=lambda t, x: 2*dw-k0_f(t)*sin(x)

        init=np.arcsin(2*dw/k0[0])

        theta=[0]*N_grid
        for i in range(N_grid):
            if i==0:
                theta[i]=init
            else:
                k1=f(t[i-1], theta[i-1])*h
                k2=f(t[i-1]+h/2, theta[i-1]+k1/2)*h
                k3=f(t[i-1]+h/2, theta[i-1]+k2/2)*h
                k4=f(t[i-1]+h, theta[i-1]+k3)*h

                theta[i]=theta[i-1]+(k1+2*k2+2*k3+k4)/6

        theta=np.array(theta)
        x0=np.sin(W*t)
        y0=np.sin(W*t+theta)

        C_0=[0]*N_grid
        for i in range(N_grid):
            if (t[i]-T/2>=0) and (t[i]+T/2<=L):
                C_0[i]=np.corrcoef(x0[i-int(p/2):i+int(p/2)], y0[i-int(p/2):i+int(p/2)])[0][1]
            else:
                C_0[i]=nan

        phi0=np.arccos(C_0)
        k_hat=np.divide(np.array([2*dw]*N_grid), np.sin(phi0))

        dmin=argrelextrema(k_hat, np.less)
        n_min=dmin[0].shape[0]
        extr=[]

        eps=0.001

        for i in range(n_min):
            if ((((i==0) and (k_hat[dmin[0][i]]<k_hat[dmin[0][i+1]]))
                or ((i==n_min-1) and (k_hat[dmin[0][i]]<k_hat[dmin[0][i-1]]))
                or ((k_hat[dmin[0][i]]<k_hat[dmin[0][i-1]]) and (k_hat[dmin[0][i]]<k_hat[dmin[0][i+1]])))
                and (dmin[0][i]>2*p) and (abs(k_hat[dmin[0][i]]-d)>eps)):
                    extr.append(dmin[0][i])

        dk=np.diff(k_hat)/h
        dk=np.nan_to_num(dk)
        #print(left, right)
        if (max(dk)>25) or ((len(extr)>1) and (extr[0]<=2*p+s) and (extr[1]>2*p+s)):
            right=s
        else:
            left=s

    stop = time.time()        
    print('-----: ', stop-start, left)
    dual_border.append(left)
    
np.savetxt('PT2407_50_20000_extr2.out', dual_border, delimiter=',')

C:\Anaconda3\lib\site-packages\scipy\signal\_peak_finding.py:68: RuntimeWarning: invalid value encountered in less
  results &= comparator(main, plus)
C:\Anaconda3\lib\site-packages\scipy\signal\_peak_finding.py:69: RuntimeWarning: invalid value encountered in less
  results &= comparator(main, minus)


-----:  116.98069071769714 12192
-----:  143.0671832561493 9808
-----:  130.1404435634613 8595
-----:  132.33856916427612 7403
-----:  127.49629235267639 6783
-----:  129.3724000453949 5813
-----:  136.8738284111023 5219
-----:  123.0710391998291 4993
-----:  135.74176406860352 4417
-----:  132.51957964897156 4372
-----:  123.96609044075012 3817
-----:  134.62069988250732 3780
-----:  129.54540967941284 3408
-----:  120.96091842651367 16838


KeyboardInterrupt: 